# LWE Grundlagen

Die Folgenden abschnitte sind zur Beschreibung und einführung in das LWE Problem. Dies ist die Grundlage für verschiedene moderne Verschlüsselungsalgorithmen wie Kyber, aber auch die basis für diverse Homomorphe Algorithmen.

Im folgenden werden 3 Arten von LWE beschrieben und implementiert
- Learning with Errors (LWE)
- Ring-LWE (RLWE)
- Module-LWE (MLWE)

Das LWE verfahren liefert dabei die Grundlagen auf denen die anderen beiden Aufbauen. Bei RLWE wird das LWE verfahren auf einen Polynom Ring übertragen und bei MLWE wird eine Matrix aus Polynom Ringen benutzt. RLWE ist somit ein Spezialfall von MLWE, bei dem eine 1x1 Matrize verwendet wird.

Zur Umsetzung von LWE muss zuerst ein Ring der ganzen zahlen modulo eines Faktors $q$ definiert werden, welcher die addition (+) und multiplikation (*) unterstützt Dies wird definiert als 
$$\mathbb{Z}_q= \mathbb{Z}/q\mathbb{Z}$$
Dies ist somit eigentlich nur die Gruppe aller ganzen Zahlen, welche kleiner als $q$ und größer gleich 0 sind. Beispielsweise $\mathbb{Z}_3 = \{0,1,2\}$

## LWE

$\chi \in \mathbb{Z}_q$ ist eine diskrete normalverteilung mit werten im Ring

Private key: $s\leftarrow \chi^{m}$. Dabei entsteht ein vektor mit länge $m$ bei dem alle werte normalverteilt aus $\mathbb{Z}_q$ stammen.

Public key:
1. erstellen einer Matrix mit gleichmäßig zufälligen werten: 
$A = \begin{bmatrix}
a_11 & \cdots  & a_1n\\
\vdots  & \ddots  & \vdots \\
a_m1 & \cdots & a_mn
\end{bmatrix} \leftarrow \mathbb{Z}_q^{n \times m} $
2. berechnen des vektors $b$ durch: $b = As+e \in \mathbb{Z}_q^{n}$ mit $e \leftarrow \chi^n$ 
3. der private schlüsse ergibt sich aus den zwei Werten: $P = (A, b)$

Zum Verschlüsseln eines binären Wertes $m \in \mathbb{Z}_2$ (entspricht den Werten $\{0, 1\}$) werden zwei Werte Berechnet:
1. $u = A^T \cdot r + e_1 \in \mathbb{Z}_q^m$
2. $v = b^T \cdot r + e_2 + (m*\left\lfloor q/2\right\rfloor) \in \mathbb{Z}_q$

Wobei $r \in \chi^n$ zusätzliche Werte sind um ein zusätzliche Verschleierung zu erzeugen und $e_1 \in \chi^m$ und $e_2 \in \chi$ sind zusätzliche Fehlerwerte. Durch diese drei werte wird jede Verschlüsselung einzigartig und somit schwerer zu entschlüsseln durch das sammeln vieler verschlüsselter Texte. 

Somit ergeben sich zwei werte, der Vektor $u$ und der skalar $v$. Die Nachricht wurde dabei auf skaliert sodas $0 \rightarrow 0$ und $1 \rightarrow \left\lfloor q/2\right\rfloor$. Wenn man sich den Ring $\mathbb{Z}_q$ als Uhr dabei vorstellt (wie es oft für modulo Rechnungen visualisiert wird), dann ist der Nachrichten Wert $0$ nun an der 12Uhr position in der Uhr und der Nachrichten Wert $1$ (ungefähr) an der 6 Uhr Position. Durch die einberechneten Fehler verändert sich der eigentliche Nachrichten Wert um ein kleines bisschen, sodass er quasi mehr in die viertel oder dreiviertel Stellung auf der Uhr zeigt.

Das entschlüsseln der Nachricht erfolgt mithilfe folgender Gleichung:
$$
\begin{align*}
m &= \left\lfloor \frac{1}{\left\lfloor q/2\right\rfloor}*(v-s^T \cdot u)\right\rceil _2 \\
  &= \left\lfloor \frac{1}{\left\lfloor q/2\right\rfloor}*(b^T \cdot r + e_2 + (m*\left\lfloor q/2\right\rfloor)-s^T \cdot (A^T \cdot r + e_1))\right\rceil _2\\
  &= \left\lfloor \frac{1}{\left\lfloor q/2\right\rfloor}*((As+e)^T \cdot r + e_2 + (m*\left\lfloor q/2\right\rfloor)-s^T A^T \cdot r - s^T e_1)\right\rceil _2\\
  &= \left\lfloor \frac{1}{\left\lfloor q/2\right\rfloor}*((As)^T \cdot r + e^Tr+ e_2 + (m*\left\lfloor q/2\right\rfloor)-(As)^T \cdot r - s^T e_1)\right\rceil _2\\
  &= \left\lfloor \frac{1}{\left\lfloor q/2\right\rfloor}*(e^Tr+ e_2 + (m*\left\lfloor q/2\right\rfloor)- s^T e_1)\right\rceil _2\\
  &= \left\lfloor \frac{e^Tr}{\left\lfloor q/2\right\rfloor}+ \frac{e_2 }{\left\lfloor q/2\right\rfloor}+ m - \frac{s^T e_1}{\left\lfloor q/2\right\rfloor}\right\rceil _2\\
  &= \left\lfloor m' \right\rceil _2\\
  &= m \in \{0,1\}
\end{align*}
$$

Bei den letzten zwei schritten wird davon ausgegangen das die Fehlerwerte nahe null sind und somit nur einen geringen einfluss auf $m$ haben. Dadurch werden diese Werte durch das runden wieder ausgeglichen und die Nachricht kommt am ende zum vorschein, welche noch in ihre ring (modulo 2) angepasst werden muss.

Damit dies funktioniert muss: $e^Tr+ e_2 - s^T e_1 < q/4$ sein. Der Fehlerterm muss somit kleiner als ein viertel q sein. Dies lässt sich erneut leicht über das Uhren beispiel erklären. Da die Zeiger am Ende auf die volle bzw halbe Stunde gerundet werden, dürfen die Zeiger nicht über die viertel oder dreiviertel stunde wander, da ansonsten zur falschen stelle gerundet wird. Dies Bewegung entspricht jeweils eine viertel Umdrehung um $q$. Somit darf der Fehlerterm nicht größer als $q/4$ sein. Aus diesem Grund muss darauf geachtet werden das $s, e, e_1, e_2, r$ nicht zu groß sind, damit dies eingehalten werden kann, weshalb diese Werte aus der Normalverteilung herausgezogen werden und nicht uniform aus $\mathbb{Z}_q$

### Beispiel Rechnung

Um die etwas abstrakten Rechnungen einfacher verständlich zu machen folgt hier eine Beispiel Rechnung.
Zuerst werden die Parameter definiert
$$
\begin{align*}
q &= 100 \\
n &= 2 \\
m &= 2 \\
\end{align*}
$$

Die Schlüsselgenerierung (KeyGen)
$$
s = \begin{bmatrix}1 \\ 2 \end{bmatrix}
A = \begin{bmatrix}56 & 77 \\ 29 & 59 \end{bmatrix}
e = \begin{bmatrix}99 \\ 1 \end{bmatrix} \\
\begin{align*}
\\
b &= As+e \\
  &= \begin{bmatrix}56 & 77 \\ 29 & 59 \end{bmatrix}\cdot \begin{bmatrix}1 \\ 2 \end{bmatrix} &+\begin{bmatrix}99 \\ 1 \end{bmatrix}\\
  &= 1 \cdot \begin{bmatrix}56 \\ 29 \end{bmatrix} + 2 \cdot \begin{bmatrix}77 \\ 59 \end{bmatrix} &+ \begin{bmatrix}99 \\ 1 \end{bmatrix} \\
  &= \begin{bmatrix}309 \\ 148 \end{bmatrix}_q \\
  &= \begin{bmatrix}9 \\ 48 \end{bmatrix} \\
\\
Sk &= s \\
Pk &= (A, b) \\
   &= (\begin{bmatrix}56 & 77 \\ 29 & 59 \end{bmatrix}, \begin{bmatrix}9 \\ 48 \end{bmatrix}) \\
\end{align*}
$$

Verschlüsselung der Nachricht $m=1$
$$
r = \begin{bmatrix}0 \\ 2 \end{bmatrix} 
e_1 = \begin{bmatrix}2 \\ 0 \end{bmatrix} 
e_2 = 99 \\
\begin{align*}
\\
u &= A^T \cdot r + e_1 \\
  &= \begin{bmatrix}56 & 77 \\ 29 & 59 \end{bmatrix}^T \cdot \begin{bmatrix}0 \\ 2 \end{bmatrix} &+ \begin{bmatrix}2 \\ 0 \end{bmatrix} \\
  &= \begin{bmatrix}56 & 29 \\ 77 & 59 \end{bmatrix} \cdot \begin{bmatrix}0 \\ 2 \end{bmatrix} &+ \begin{bmatrix}2 \\ 0 \end{bmatrix} \\
  &= 0\cdot \begin{bmatrix}56 \\ 77 \end{bmatrix} + 2 \cdot \begin{bmatrix}29 \\ 59 \end{bmatrix} &+ \begin{bmatrix}2 \\ 0 \end{bmatrix} \\
  &= \begin{bmatrix}60 \\ 118 \end{bmatrix}_q \\
  &= \begin{bmatrix}60 \\ 18 \end{bmatrix} \\
\\
v &= b^T \cdot r + e_2 + (m*\left\lfloor q/2\right\rfloor) \\
  &= \begin{bmatrix}9 \\ 48 \end{bmatrix}^T \cdot \begin{bmatrix}0 \\ 2 \end{bmatrix} + 99 + 1 \cdot \left\lfloor 100/2\right\rfloor \\
  &= \begin{bmatrix}9 & 48 \end{bmatrix} \cdot \begin{bmatrix}0 \\ 2 \end{bmatrix} + 99 + 50 \\
  &= 9 \cdot 0 +48 \cdot 2 + 99 + 50 \\
  &= 245_q \\
  &= 45 \\
\end{align*}

C = (u, v)
$$

Entschlüssel der verschlüsselten Nachricht $C$
$$
\begin{align*}
m &= \left\lfloor \frac{1}{\left\lfloor q/2\right\rfloor} *(v-s^T \cdot u)\right\rceil _2 \\
  &= \left\lfloor \frac{1}{\left\lfloor 100/2\right\rfloor} * (45-\begin{bmatrix}1 \\ 2 \end{bmatrix}^T \cdot \begin{bmatrix}60 \\ 18 \end{bmatrix})\right\rceil _2 \\
  &= \left\lfloor \frac{1}{50} * (45-\begin{bmatrix}1 & 2 \end{bmatrix} \cdot \begin{bmatrix}60 \\ 18 \end{bmatrix})\right\rceil _2 \\
  &= \left\lfloor \frac{1}{50} * (45-(60 \cdot 1 + 18 \cdot 2))\right\rceil _2 \\
  &= \left\lfloor \frac{1}{50} * (-33)_q\right\rceil _2 \\
  &= \left\lfloor \frac{1}{50} * 67\right\rceil _2 \\
  &= \left\lfloor \frac{67}{50}\right\rceil _2 \\
  &= 1 \\
\end{align*}
$$

Somit konnte die Originale Nachricht wiederhergestellt werden

### Beispiel Code

In [426]:
import numpy as np
from numpy.random import randint, normal

def sample_error(n, q, sigma=1.0):
  """Samples error vector from an approximated discrete Gaussian distribution.
  """
  # Sample from continuous Gaussian with mean 0 and std dev sigma
  error = np.round(normal(loc=0, scale=sigma, size=n)).astype(int) % q 

  # if its the zero vector, try again
  if sum(error) == 0:
    return sample_error(n, q, sigma)
  return error

# Parameters
q = 100
t = 2 # The message ring
q_half = np.floor(q/2)
n = 2
m = 2

# Creating Privat and Secret Key
s = sample_error(m, q)
A = randint(0, q, (n, m))
e = sample_error(n, q)
b = (A@s+e) % q

print(f"""Secret Key: {s}
Private Key:
      A={A},
      b={b}
Generated with error: {e}""")

Secret Key: [1 2]
Private Key:
      A=[[56 77]
 [29 59]],
      b=[ 9 48]
Generated with error: [99  1]


In [427]:
message = randint(0, t) # random 0 or 1 as message
r = sample_error(n, q)
e1 = sample_error(m, q)
e2 = sample_error(1, q)
u = (A.T@r+e1) % q
v = (b.T@r+e2+message*q_half) % q

print(f"""Encrypting message: {message}
using:
    r={r},
    e1={e1}
    e2={e2}
Resulting in:
    u={u}
    v={v}
""")
decrypt = np.round((1/q_half)*((v-(s.T@u)) % q)) % t
print(f"""Decrypts into message: {decrypt}""")

Encrypting message: 1
using:
    r=[0 2],
    e1=[2 0]
    e2=[99]
Resulting in:
    u=[60 18]
    v=[45.]

Decrypts into message: [1.]
